## Stuff I Did This Trimester

..definitely not nothing...

### A small overview of my role in the team
- Lead morale increaser
- Documenter
- Bugfixer
- Lead artist (background)

## Two Player

- created 1st and 2nd versions of the final fix that enabled two player
- setup sprites for use

## Documentation

- I was on the "notebook creating" sub-team
- Assisted Jim in creating lesson notebooks for our API and two-player lessons
- Created documentation of our changes to the adventure game using <b><i>Tailwind CSS<i><b>

## Debugging

- Debugging is something that nobody wants to do
- And so naturally I learned how to interpret these alien error messages
- I eventually found it easier to look at the errors, use the debugger, and comb through files myself rather than ask any LLM

- However, I still have much to learn (this breakpoint stuff confuses me)

## Unrelated things

- Worked on a passion project not using JS
- Used YOLO for object detection and wrote a python file that would interpret these detections and display them on the screen.
- It would also display and output coordinates for use with some kind of grabbing apparatus (didn't get to this part b/c it would cost a lot to build a robot arm)


## Unrelated things 2

- gained appreciation for game devs (man this coding stuff is hard)
- gained appreciation for artists (man this drawing stuff is hard)